In [3]:
import pandas as pd
import numpy as np
from scipy.optimize import bisect
from scipy.stats import norm

file_path = './SPX_hedging.csv'
data = pd.read_csv(file_path).dropna()
data['TTM_years'] = data['TTM']/252

In [4]:
data.describe()

,ID,Days until next hedge,S,Dividend,C_BS,D_BS,C_mkt,D_Blm,R,TTM,Moneyness,D_Optimal,Target,K,TTM_years
count,84.0,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.0,84.000000
mean,1.0,1.416667,5714.541786,1.346754,298.537744,0.588672,313.172619,0.606857,4.805952,169.595238,14.511905,0.055930,-0.532742,5700.0,0.672997
std,0.0,0.809705,199.801300,0.046879,92.721205,0.119188,93.933201,0.118693,0.106919,34.656357,199.854254,5.289785,5.295591,0.0,0.137525
min,1.0,1.000000,5186.330000,1.277400,106.223538,0.291429,111.850000,0.312000,4.590720,109.000000,-514.000000,-39.819644,-40.533411,5700.0,0.432540
25%,1.0,1.000000,5596.780000,1.312575,235.628712,0.525178,255.462500,0.549000,4.759908,140.750000,-103.250000,0.423916,-0.145064,5700.0,0.558532
50%,1.0,1.000000,5725.530000,1.337250,295.365778,0.591041,311.050000,0.609000,4.780625,169.500000,25.500000,0.583627,-0.006258,5700.0,0.672619
75%,1.0,1.000000,5851.895000,1.372475,366.632106,0.669142,380.075000,0.685250,4.883605,198.250000,151.750000,0.772258,0.156792,5700.0,0.786706
max,1.0,3.000000,6047.150000,1.481600,465.237571,0.798269,483.900000,0.816000,5.014990,228.000000,347.000000,8.794966,8.216118,5700.0,0.904762


In [5]:
# Black-Scholes call option pricing formula
def black_scholes_call_price(S, K, T, r, sigma):
    if T <= 0:
        return max(S - K, 0)  # Payoff at maturity
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Function to calculate implied volatility
def implied_volatility(C_mkt, S, K, T, r):
    def objective(sigma):
        price = black_scholes_call_price(S, K, T, r, sigma)
        # print("price:", price)
        output = price - C_mkt
        # print(output)
        return output
    try:
        return bisect(objective, 1e-6, 10)  # Expanded range
    except ValueError:
        return np.nan

# Function to calculate call Delta using the Black-Scholes model
def black_scholes_call_delta(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return norm.cdf(d1)

# Crank-Nicolson method for solving the Black-Scholes PDE
def crank_nicolson(S, K, r, T, sigma, N=100, M=100):
    dt = T / N  # Time step size
    S_min = 0
    S_max = 2 * S 
    # S_max = S * np.exp((r - (sigma**2)/2) * T + 3 * sigma * np.sqrt(T))
    # S_min = S * np.exp((r - (sigma**2)/2) * T - 3 * sigma * np.sqrt(T
    dS = S_max/M
    # print(f"S: {S}, S_min: {S_min}, S_max: {S_max}")
    
    # Initialize the price grid
    S_grid = np.linspace(S_min, S_max, M + 1)
    # print(S_grid)
    V = np.maximum(S_grid - K, 0)  # Option value at maturity

    # Coefficinents for Crank-Nicolson
    alpha = 0.25 * dt * ((sigma**2 * S_grid**2) / (dS**2) - r * S_grid / dS)
    beta = -0.5 * dt * (sigma**2 * S_grid**2 / (dS**2) + r)
    gamma = 0.25 * dt * ((sigma**2 * S_grid**2) / (dS**2) + r * S_grid / dS)
    
    # Implicit matrix
    A = np.zeros((M-1, M-1))
    B = np.zeros((M-1, M-1))
    for i in range(1, M):
        if i > 1:
            A[i-1, i-2] = -alpha[i]
            B[i-1, i-2] = alpha[i]
        A[i-1, i-1] = 1 - beta[i]
        B[i-1, i-1] = 1 + beta[i]
        if i < M-1:
            A[i-1, i] = -gamma[i]
            B[i-1, i] = gamma[i]
    
    # A_matrix = pd.DataFrame(A)
    # B_matrix = pd.DataFrame(B)
    # print(f"A: {A_matrix.head()} \n B: {B_matrix.head()}")
    
    # Time stepping
    for _ in range(N):
        V_inner = V[1:M]
        # print(f'V_inner at t+1: {V_inner}')
        V_inner = np.linalg.solve(A, B @ V_inner)
        V[1:M] = V_inner
        # print(f'V_inner at t: {V_inner}')
        V[-1] = S_max - K * np.exp(-r * (T - _ * dt))  # Boundary condition at S -> infinity
        
    # print(f"S: {S}, S_grid: {S_grid}, V: {V}")

    return np.interp(S, S_grid, V)


C_mkt = 2.59
S = 100
K = 110
T = 0.5
r = 0.03
sigma = 0.2

print(crank_nicolson(S, K, r, T, sigma))
# # Calculate implied volatility
# sigma_imp = implied_volatility(C_mkt, S, K, T, r)
# print(f"Implied Volatility: {sigma_imp:.4f}")
# More accurate using smaller grid, maybe not small enough discretization (stock price step discretization)
# linspace smin and smax
# Time discretization should be smaller

2.60257880316775


In [10]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        print(f"{func.__name__} executed in {end_time - start_time:.6f} seconds")
        return result
    return wrapper

@timer
def test_delta_hedge(data):
    # Initialize strategy variables
    position = 0  # Current asset position
    cash = 0      # Cash balance
    portfolio_values = []  # Store portfolio values over time
    predicted_prices = []  # Store predicted option prices
    
    # Iterate over each row of the data to implement the hedging strategy
    for i in range(len(data)):
        S = data['S'][i]  # Current stock pricex
        K = data['K'][i]  # Calculate strike price based on Moneyness
        T = data['TTM_years'][i] 
        r = (data['R'][i] - data['Dividend'][i]) / 100  # Convert interest rate to decimal
        C_mkt = data['C_mkt'][i]  # Market option price
    
        # Calculate implied volatility
        sigma = implied_volatility(C_mkt, S, K, T, r)
        # Skip this row if implied volatility could not be calculated
        if np.isnan(sigma):
            print(f'IV could not be derived from row {i}')
            continue
    
        # Calculate Delta using Crank-Nicolson if needed
        epsilon = 1e-4  # Small change for Delta calculation
        V_up = crank_nicolson(S * (1 + epsilon), K, r, T, sigma, 100)
        V_down = crank_nicolson(S * (1 - epsilon), K, r, T, sigma, 100)
        delta = (V_up - V_down) / (S * 2 * epsilon)
    
        # Calculate predicted option price using Crank-Nicolson
        predicted_price = crank_nicolson(S, K, r, T, sigma)
        predicted_prices.append(predicted_price)
    
        # Determine the target position based on Delta
        target_position = -delta
        position_change = target_position - position
    
        # Update cash and asset position
        cash -= position_change * S
        position = target_position
    
        # Record portfolio value
        portfolio_values.append(position * S + cash)
    return predicted_prices, portfolio_values

In [11]:
portfolio_values, predicted_prices = test_delta_hedge(data)

# Convert portfolio values to a DataFrame for further analysis
portfolio_values = pd.DataFrame(portfolio_values, columns=['Portfolio Value'])

# Print predicted prices
predicted_prices_df = pd.DataFrame(predicted_prices, columns=['Predicted Price'])
print(predicted_prices_df.head())
print("---------------")
print(data.head()['C_mkt'])
print("---------------")
print(portfolio_values.head())

# Calculate risk metrics
portfolio_values['Returns'] = portfolio_values['Portfolio Value'].pct_change().dropna()

returns = portfolio_values.loc[
    (pd.notnull(portfolio_values['Returns'])) & (np.isfinite(portfolio_values['Returns'])),
    'Returns'
]

# print(portfolio_values.head())
volatility = returns.std()  # Calculate return volatility, first row is NaN (no change) second row is -inf
# print(volatility)
cumulative_returns = (1 + returns).cumprod()
drawdown = cumulative_returns.cummax() - cumulative_returns  # Calculate drawdown
max_drawdown = drawdown.max()
# # Output risk metrics
print("return volatility", volatility, "max drawdown:", max_drawdown)

test_delta_hedge executed in 10.093613 seconds
   Predicted Price
0         0.000000
1        -1.941488
2        -0.593796
3        -4.546418
4        -5.523876
---------------
0    147.20
1    125.90
2    111.85
3    153.75
4    145.55
Name: C_mkt, dtype: float64
---------------
   Portfolio Value
0       146.802597
1       125.990089
2       111.747390
3       153.817292
4       145.536183
return volatility 0.09653593246369267 max drawdown: 0.8520130792690124
